In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')
sample_sub = pd.read_csv('../input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.info()

No null values, all features except f_27 are numerical 

In [ ]:
train['f_27'].nunique()

In [ ]:
train['f_27'].value_counts()

In [ ]:
corr_df = train.drop('f_27',axis=1)

In [ ]:
corr = corr_df.corr()

plt.figure(figsize=(16,9))
sns.heatmap(corr,vmin=-0.5,vmax=1)

From this heatmap it can be seen that there are some correlation among the features, 

In [ ]:
plot_df = corr_df.drop(['id','target'],axis=1)

i=0
while i < len(plot_df.columns):
    fig, axs = plt.subplots(1,3,figsize=(10, 3))
    sns.histplot(data=plot_df,x= plot_df[plot_df.columns[i]],color="skyblue",ax=axs[0])
    axs[0].set_title(plot_df.columns[i])
    #axs[0].set_xlim(xmin=-3.5,xmax=3.5)
    sns.histplot(data=plot_df,x=plot_df[plot_df.columns[i+1]],color="skyblue",ax=axs[1])
    axs[1].set_title(plot_df.columns[i+1])
    #axs[1].set_xlim(xmin=-3.5,xmax=3.5)
    sns.histplot(data=plot_df,x=plot_df[plot_df.columns[i+2]],color="skyblue",ax=axs[2])
    axs[2].set_title(plot_df.columns[i+2])
    
    i+=3
    plt.tight_layout()
    plt.show()

In [ ]:
train['target'].value_counts()

In [ ]:
y_train = train['target']
train = train.drop('target',axis=1)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

f_27 = le.fit_transform(train['f_27'])
train['f_27'] = f_27

In [ ]:
from sklearn.feature_selection import mutual_info_regression

mi_scores = mutual_info_regression(train, y_train)
mi_scores = pd.Series(mi_scores, name="MI_score", index=train.columns)
mi_scores = mi_scores.sort_values(ascending=False)
df_mi_scores1 = pd.DataFrame(mi_scores).reset_index().rename(columns={'index':'feature'})
df_mi_scores1

In [ ]:
import eli5
from catboost import CatBoostClassifier
from eli5.sklearn import PermutationImportance

model1 = CatBoostClassifier(logging_level='Silent', random_state=42, eval_metric='AUC',loss_function='Logloss').fit(train,y_train)

perm1 = PermutationImportance(model1, random_state=1).fit(train, y_train)
eli5.show_weights(perm1, feature_names = train.columns.tolist(), top=None)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

norm_df = StandardScaler().fit_transform(train)

pca = PCA(n_components=6)
principalComponents = pca.fit_transform(norm_df)
pc_df = pd.DataFrame(data = principalComponents
             , columns = ['pc1','pc2','pc3','pc4','pc5','pc6'])

In [ ]:
train2 = pd.concat([pc_df, train], axis = 1)

In [ ]:
model2 = CatBoostClassifier(logging_level='Silent', random_state=42, eval_metric='AUC',loss_function='Logloss').fit(train2,y_train)

perm1 = PermutationImportance(model2, random_state=1).fit(train2, y_train)
eli5.show_weights(perm1, feature_names = train2.columns.tolist(), top=None)